# Fair and Robust Sample Selection on the Synthetic Dataset
## With Group-Targeted Label Flipping

#### This Jupyter Notebook simulates the proposed fair and robust sample selection on the synthetic data.
#### We use two fairness metrics: equalized odds and demographic parity.

## Import libraries

In [2]:
import sys, os
import numpy as np
import math
import random
import itertools
import copy

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch

from models import LogisticRegression, weights_init_normal, test_model
from FairRobustSampler import FairRobust, CustomDataset

from argparse import Namespace

import warnings
warnings.filterwarnings("ignore")

In [1]:
import torch
torch.cuda.is_available()

False

## Load and process the data
In the synthetic_data directory, there are a total of 11 numpy files including training data (both clean and noisy), validation data, and test data. Note that the validation data is utilized for another method in the paper (i.e., FR-Train), so the data is not used in this program.

In [4]:
xz_train = np.load('./synthetic_data/xz_train.npy')
y_train = np.load('./synthetic_data/y_train.npy')
z_train = np.load('./synthetic_data/z_train.npy')

y_noise = np.load('./synthetic_data/y_noise_grouptarget.npy') # Labels with the group-targeted label flipping (details are in the paper)
poi_ratio = 0.1

xz_test = np.load('./synthetic_data/xz_test.npy')
y_test = np.load('./synthetic_data/y_test.npy') 
z_test = np.load('./synthetic_data/z_test.npy')

xz_train = torch.FloatTensor(xz_train)
y_train = torch.FloatTensor(y_train)
z_train = torch.FloatTensor(z_train)

y_noise = torch.FloatTensor(y_noise)

xz_test = torch.FloatTensor(xz_test)
y_test = torch.FloatTensor(y_test)
z_test = torch.FloatTensor(z_test)

In [5]:
if torch.cuda.is_available():
    xz_train = xz_train.cuda()
    y_noise = y_noise.cuda()
    y_train = y_train.cuda()
    z_train = z_train.cuda()
    
    xz_test = xz_test.cuda()
    y_test = y_test.cuda()
    z_test = z_test.cuda()

In [6]:
print("---------- Number of Data ----------" )
print(
    "Train data : %d, Test data : %d "
    % (len(y_train), len(y_test))
)       
print("------------------------------------")

---------- Number of Data ----------
Train data : 2000, Test data : 1000 
------------------------------------


## Training function

In [7]:
def run_epoch(model, train_features, labels, optimizer, criterion):
    """Trains the model with the given train data.

    Args:
        model: A torch model to train.
        train_features: A torch tensor indicating the train features.
        labels: A torch tensor indicating the true labels.
        optimizer: A torch optimizer.
        criterion: A torch criterion.

    Returns:
        loss values.
    """
    
    optimizer.zero_grad()

    label_predicted = model.forward(train_features)
    loss  = criterion((F.tanh(label_predicted.squeeze())+1)/2, (labels.squeeze()+1)/2)
    loss.backward()

    optimizer.step()
    
    return loss.item()

# 1. Fair and Robust Sample Selection w.r.t. Equalized Odds
### The results are in the experiments of the paper.

In [8]:
full_tests = []

parameters = Namespace(warm_start=100, tau=1-poi_ratio, alpha = 0.001, batch_size = 100)

# Set the train data
train_data = CustomDataset(xz_train, y_noise, z_train)

seeds = [0,1,2,3,4]

for seed in seeds:
    
    print("< Seed: {} >".format(seed))
    
    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------
    
   #  model = LogisticRegression(3,1).cuda()

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    # ---------------------
    #  Define FairRobust and DataLoader
    # ---------------------

    sampler = FairRobust (model, train_data.x, train_data.y, train_data.z, target_fairness = 'eqodds', parameters = parameters, replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    # ---------------------
    #  Model training
    # ---------------------
    for epoch in range(450):
        print(epoch, end="\r")
        
        tmp_loss = []
        
        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)
            
        losses.append(sum(tmp_loss)/len(tmp_loss))
        

    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, EO disparity: {}".format(tmp_test['Acc'], tmp_test['EqOdds_diff']))
    print("----------------------------------------------------------------------")

< Seed: 0 >


AssertionError: Torch not compiled with CUDA enabled

In [7]:
tmp_acc = []
tmp_eo = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_eo.append(full_tests[i]['EqOdds_diff'])

print("Test accuracy (avg): {}".format(sum(tmp_acc)/len(tmp_acc)))
print("EO disparity  (avg): {}".format(sum(tmp_eo)/len(tmp_eo)))

Test accuracy (avg): 0.7260000228881835
EO disparity  (avg): 0.040289772577944014


# 2. Fair and Robust Sample Selection w.r.t. Demographic Parity
### The results are in the experiments of the paper.

In [8]:
full_tests = []

parameters = Namespace(warm_start=100, tau=1-poi_ratio, alpha = 0.001, batch_size = 100)

# Set the train data
train_data = CustomDataset(xz_train, y_noise, z_train)

seeds = [0,1,2,3,4]

for seed in seeds:
    
    print("< Seed: {} >".format(seed))
    
    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------
    
    model = LogisticRegression(3,1).cuda()

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    # ---------------------
    #  Define FairRobust and DataLoader
    # ---------------------

    sampler = FairRobust (model, train_data.x, train_data.y, train_data.z, target_fairness = 'dp', parameters = parameters, replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    # ---------------------
    #  Model training
    # ---------------------
    for epoch in range(500):
        print(epoch, end="\r")
        
        tmp_loss = []
        
        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)
            
        losses.append(sum(tmp_loss)/len(tmp_loss))
        
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, DP disparity: {}".format(tmp_test['Acc'], tmp_test['DP_diff']))
    print("----------------------------------------------------------------------")

< Seed: 0 >
  Test accuracy: 0.7220000624656677, DP disparity: 0.020555555555555605
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.7210000157356262, DP disparity: 0.03270707070707074
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.718000054359436, DP disparity: 0.05906060606060609
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.718000054359436, DP disparity: 0.042858585858585874
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.7200000286102295, DP disparity: 0.03980808080808085
----------------------------------------------------------------------


In [9]:
tmp_acc = []
tmp_dp = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_dp.append(full_tests[i]['DP_diff'])

print("Test accuracy (avg): {}".format(sum(tmp_acc)/len(tmp_acc)))
print("DP disparity  (avg): {}".format(sum(tmp_dp)/len(tmp_dp)))

Test accuracy (avg): 0.7198000431060791
DP disparity  (avg): 0.03899797979797983
